![banner](../banner.jpg)

***[Workshop 5](about) - Training course in data analysis for genomic surveillance of African malaria vectors***

---

# Module 4 - Heterozygosity and runs of homozygosity

__Theme: analysis__

In this module we are going to investigate heterozygosity and homozygosity using plots across the genome and runs of homozygosity (ROH). These analyses can help us understand more about the demographic history of a mosquito population. 

## Learning Objectives
At the end of this module, you will be able to:
- Understand and be able to calculate heterozygosity
- Plot and interpret the heterozygosity across the genome for individual samples
- Explain ROH analysis
- Run, plot and interpret ROH

## Lecture

### English

In [31]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/D2mHfxbJQfc" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

### Français

In [32]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/u_EWYjPC8iU" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

## Setup
First, let’s begin by installing and importing some Python packages, and configuring access to _Anopheles_ genomic data from the MalariaGEN Ag3.0 data resource.

In [1]:
!pip install -q malariagen_data

In [2]:
import malariagen_data

In [3]:
ag3 = malariagen_data.Ag3()
ag3

<MalariaGEN Ag3 API client>
Storage URL             : gs://vo_agam_release/
Data releases available : 3.0
Results cache           : None
Cohorts analysis        : 20220608
Species analysis        : aim_20220528
Site filters analysis   : dt_20200416
Software version        : malariagen_data 6.0.0
Client location         : Iowa, US (Google Cloud)
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact data@malariagen.net. For API documentation see 
https://malariagen.github.io/vector-data/ag3/api.html

Check your __Client location__ (virtual machine or VM) is located near to the data in the US. If this isn't the case, use `Runtime - Disconnect and delete runtime` from the menu, then re-run the notebook to generate a new VM.

## Heterozygosity in wild populations
Mosquitoes are diploid, so an individual carries two copies of each autosome (non-sex chromosome), one from each parent. This means that each autosomal nucleotide position in the genome can be homozygous, where the nucleotide from each chromosome is the same (e.g., A/A, C/C, T/T or G/G), or heterozygous, where the nucleotides differ (e.g., A/T, C/G, etc.).

In the figure below, we see that at a single genomic position where both parents are heterozygous, the probability of heterozygous offspring is 0.5 (1/2). If only one of the parents in a mating was heterozygous then the probability of heterozygous offspring would decrease to 0.25 (1/4). 

<img src="https://storage.googleapis.com/vo_agam_release/reference/training-images/workshop-5/heterozygosity.png"/>

In [4]:
# %%HTML
# <iframe frameborder="0" style="width:100%;height:700px;" src="https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&title=heterozygosity.drawio#Uhttps%3A%2F%2Fdrive.google.com%2Fuc%3Fid%3D1EtDxxblllykmAYnbONba717PlvZTdnhC%26export%3Ddownload"></iframe>


Analysing heterozygosity across the genome of an individual can give an indication how __inbred__ an individual is. If heterozygosity is generally low, or there are regions with depressed heterozygosity, then the parents of the individual were likely __closely related__. This can happen by chance of course, but if we find that many individuals from a given population have low heterozygosity, then the population may be __small__ or have recently been through a __bottleneck__ (we'll come back to that). 
  
Let's plot heterozygosity across the genome of some of our mosquito samples and learn how to interpret the figures.

First, we should load the sample metadata to help us pick some samples.

In [5]:
sample_df = ag3.sample_metadata()
sample_df.query("country == 'Burkina Faso'").head()

Load sample metadata:   0%|          | 0/28 [00:00<?, ?it/s]

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,...,aim_species,country_iso,admin1_name,admin1_iso,admin2_name,taxon,cohort_admin1_year,cohort_admin1_month,cohort_admin2_year,cohort_admin2_month
81,AB0085-Cx,BF2-4,Austin Burt,Burkina Faso,Pala,2012,7,11.151,-4.235,F,...,gambiae,BFA,Hauts-Bassins,BF-09,Houet,gambiae,BF-09_gamb_2012,BF-09_gamb_2012_07,BF-09_Houet_gamb_2012,BF-09_Houet_gamb_2012_07
82,AB0086-Cx,BF2-6,Austin Burt,Burkina Faso,Pala,2012,7,11.151,-4.235,F,...,gambiae,BFA,Hauts-Bassins,BF-09,Houet,gambiae,BF-09_gamb_2012,BF-09_gamb_2012_07,BF-09_Houet_gamb_2012,BF-09_Houet_gamb_2012_07
83,AB0087-C,BF3-3,Austin Burt,Burkina Faso,Bana Village,2012,7,11.233,-4.472,F,...,coluzzii,BFA,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07
84,AB0088-C,BF3-5,Austin Burt,Burkina Faso,Bana Village,2012,7,11.233,-4.472,F,...,coluzzii,BFA,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07
85,AB0089-Cx,BF3-8,Austin Burt,Burkina Faso,Bana Village,2012,7,11.233,-4.472,F,...,coluzzii,BFA,Hauts-Bassins,BF-09,Houet,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07


For our first heterozygosity plot, we'll pick a wild-caught _Anopheles gambiae_ sample from Burkina Faso, "AB0085-Cx".

We can use the `plot_heterozygosity()` function to visualise heterozygosity in a single sample over a chromosome arm. Let's have a look at what parameters are required by the`plot_heterozygosity()` function.

In [6]:
ag3.plot_heterozygosity?

The *Anopheles gambiae* genome is relatively large, e.g., there are 53 million nucleotide positions on the 3R chromosome arm. Plotting millions of points is just not practical either computationally or when it comes to interpretation. Consequently we use a __windowed__ approach, where we define a window size in number of sites (genomic positions), then move this window along the chromosome arm, taking the mean of our statistic (here, heterozygosity) in every window. This gives us a summary of data that is easier to interpret.
  
Here we'll use a window size of 10,000 sites. As we know that this sample is _Anophele gambiae_ (see "taxon" column in metadata), we should use the __"gamb_colu" site mask__ as this will give us just the sites that pass site filters, _i.e._ sites we are confident in calling genotypes, for _An. gambiae_ or _An. coluzzii_. 

In [7]:
ag3.plot_heterozygosity(
    sample="AB0085-Cx", 
    region="3R", 
    site_mask="gamb_colu", 
    window_size=10_000,
);

Compute heterozygous genotypes:   0%|          | 0/350 [00:00<?, ?it/s]

Each marker shows the heterozygosity in a single window, where heterozygosity is the fraction of sites with a heterozygous genotype call.

The heterozygosity we see for individual AB0085-Cx is typical of many _Anopheles_ samples, with most windows having a heterozygosity of 0.015 (1.5%). We see just a few windows with very low heterozygosity along most of the contig, with a drop at the telomeric end (left) and a decline at the centromeric end (right). 

Note that heterozygosity is typically lower in all samples in both telomeric and pericentromeric regions, because of reduced rates of recombination.

How does this sample's heterozygosity, across the 3R contig, compare with one of the lab colony samples?

## Heterozygosity in lab colonies

A lab colony is usually founded by taking a small number of gravid females from the wild and rearing the offspring in cage. Genetically this differs from wild mosquito populations in two major ways.

### 1. Population bottleneck.

As the lab colony is founded by only a few wild individuals, it is highly unlikely that all the genetic diversity found in the wild population will be represented in the colony. This is known as a __founder effect__ which is a kind of __population bottleneck__. The term bottleneck refers to the idea that the original population is squeezed through the slim neck of a bottle, so only some individuals/genetic-variants make it through to the new smaller population.

### 2. Population size.

In the wild, a mosquito population may consist of millions of individuals and even after a severe population bottleneck, like an island founding, it would be expected that the population would over time become large again. However, in a colony, the population size is kept artifically small to stop cage overcrowding and associated drops in colony fitness. A genetic side-effect of this very small population size is an increase in the power of __genetic drift__. Genetic drift is the random fluctuations in allele frequencies due to chance, and in small populations it has a much stronger effect than in large populations. This means that it is much more likely for alleles to become fixed (100%) in a colony than in a natural population, and so over a few generations the already low heterozygosity (due to bottleneck) is likely to drop even further.

<img src="https://storage.googleapis.com/vo_agam_release/reference/training-images/workshop-5/bottleneck.png"/>

In [8]:
# %%HTML
# <iframe frameborder="0" style="width:100%;height:700px;" src="https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&title=bottleneckdrawio#Uhttps%3A%2F%2Fdrive.google.com%2Fuc%3Fid%3D1Eq1t9Lch5NZ-buO07zKXAtxSFFAAmaNP%26export%3Ddownload"></iframe>

Let's choose a lab colony sample from the lab colony cross metadata to look at.

In [9]:
crosses_df = ag3.cross_metadata()
crosses_df.query("role == 'parent'")

,cross,sample_id,father_id,mother_id,sex,role
0,18-5,AD0142-C,NaN,NaN,F,parent
1,18-5,AD0143-C,NaN,NaN,M,parent
22,29-2,AD0231-C,NaN,NaN,F,parent
23,29-2,AD0232-C,NaN,NaN,M,parent
44,36-9,AD0254-C,NaN,NaN,F,parent
45,36-9,AD0255-C,NaN,NaN,M,parent
66,37-3,AD0277-C,NaN,NaN,F,parent
67,37-3,AD0278-C,NaN,NaN,M,parent
88,42-4,AD0305-C,NaN,NaN,F,parent
89,42-4,AD0306-C,NaN,NaN,M,parent


Let's look at sample AD0305-C which was the mother in cross 42-4.

In [10]:
ag3.plot_heterozygosity(
    sample="AD0305-C", 
    region="3R", 
    site_mask="gamb_colu", 
    window_size=10_000,
);

Compute heterozygous genotypes:   0%|          | 0/350 [00:00<?, ?it/s]

If we compare this to our wild-caught sample we can see that although some of the windows still have heterozygosity ~1.5%, there are also some large regions of the chromosome where heterozygosity is almost zero. These are known as **"runs of homozygosity"**.

Let's also look at sample AD0306-C which was the father of cross 42-4.

In [11]:
ag3.plot_heterozygosity(
    sample="AD0306-C", 
    region="3R", 
    site_mask="gamb_colu", 
    window_size=10_000,
);

Compute heterozygous genotypes:   0%|          | 0/350 [00:00<?, ?it/s]

This sample has even more runs of homozygosity. Hardly any windows remain with the normal level of heterozygosity. This is a symptom of significant inbreeding.

Let's now compare the heterozygosity of these two parents with the heterozygosity in their progeny.


In [12]:
crosses_df.query("cross == '42-4' and role == 'progeny'")

,cross,sample_id,father_id,mother_id,sex,role
90,42-4,AD0309-C,AD0306-C,AD0305-C,M,progeny
91,42-4,AD0310-C,AD0306-C,AD0305-C,M,progeny
92,42-4,AD0311-C,AD0306-C,AD0305-C,M,progeny
93,42-4,AD0312-C,AD0306-C,AD0305-C,M,progeny
94,42-4,AD0313-C,AD0306-C,AD0305-C,M,progeny
95,42-4,AD0314-C,AD0306-C,AD0305-C,M,progeny
96,42-4,AD0315-C,AD0306-C,AD0305-C,M,progeny
97,42-4,AD0316-C,AD0306-C,AD0305-C,F,progeny
98,42-4,AD0317-C,AD0306-C,AD0305-C,M,progeny
99,42-4,AD0318-C,AD0306-C,AD0305-C,M,progeny


Let's pick one of the progeny, AD0309-C.

In [13]:
ag3.plot_heterozygosity(
    sample="AD0309-C", 
    region="3R", 
    site_mask="gamb_colu", 
    window_size=10_000,
);

Compute heterozygous genotypes:   0%|          | 0/350 [00:00<?, ?it/s]

This sample has normal levels of heterozygosity, similar to wild samples. So how did this happen if both the parents showed signs of inbreeding?

The progeny of cross 42-4 are the result of mating a female from one lab colony to a male from a different lab colony. This F1 pairing of two different bottlenecked populations should restore heterozygosity as these cross progeny are effectively __outbred__, i.e., their parents are not closely related.

This is roughly analogous to breeding between two different pedigree breeds, such as dog breeds for example. Each of the parents may be relatively inbred due to many generations of mating within a small population, but the two different breeds are not closely related, and so their progeny show normal levels of heterozygosity.

## Heterozygosity in island populations

As we have learnt, taking a single individual (or small number of individuals) from the wild to seed a lab colony results in a reduction in heterozygosity as it results in a __bottleneck__ event. A similar effect can also occur naturally when an animal populates an island from the mainland. 
  
An island population will likely be founded with only a fraction of the mainland population diversity, plus island populations would tend to be smaller than those on the mainland, increasing __inbreeding__ and the effect of __drift__ (though not to the levels seen in the small lab colonies).
  
In Ag3.0 we have samples from the oceanic island Mayotte, ~1000km from mainland Africa and a similar distance from Madagascar. Due to these distances, it is likely that the _Anopheles gambiae_ mosquito population was founded by just a few individuals. Let's look at heterozygosity in a Mayotte sample.

In [14]:
sample_df.query("country == 'Mayotte'").head()

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,...,aim_species,country_iso,admin1_name,admin1_iso,admin2_name,taxon,cohort_admin1_year,cohort_admin1_month,cohort_admin2_year,cohort_admin2_month
1050,AP0007-C,60,Igor Sharakhov,Mayotte,Mtsamboro Forest Reserve,2011,-1,-12.703,45.081,F,...,gambiae,FRA,Mayotte,FR-YT,Mayotte,gambiae,FR-YT_gamb_2011,FR-YT_gamb_2011,FR-YT_Mayotte_gamb_2011,FR-YT_Mayotte_gamb_2011
1051,AP0021-C,92,Igor Sharakhov,Mayotte,Karihani Lake,2011,-1,-12.797,45.122,F,...,gambiae,FRA,Mayotte,FR-YT,Mayotte,gambiae,FR-YT_gamb_2011,FR-YT_gamb_2011,FR-YT_Mayotte_gamb_2011,FR-YT_Mayotte_gamb_2011
1052,AP0019-C,88,Igor Sharakhov,Mayotte,Mtsanga Charifou,2011,-1,-12.991,45.156,M,...,gambiae,FRA,Mayotte,FR-YT,Mayotte,gambiae,FR-YT_gamb_2011,FR-YT_gamb_2011,FR-YT_Mayotte_gamb_2011,FR-YT_Mayotte_gamb_2011
1053,AP0020-C,78,Igor Sharakhov,Mayotte,Mtsanga Charifou,2011,-1,-12.991,45.156,M,...,gambiae,FRA,Mayotte,FR-YT,Mayotte,gambiae,FR-YT_gamb_2011,FR-YT_gamb_2011,FR-YT_Mayotte_gamb_2011,FR-YT_Mayotte_gamb_2011
1054,AP0009-C,62,Igor Sharakhov,Mayotte,Combani,2011,-1,-12.779,45.143,M,...,gambiae,FRA,Mayotte,FR-YT,Mayotte,gambiae,FR-YT_gamb_2011,FR-YT_gamb_2011,FR-YT_Mayotte_gamb_2011,FR-YT_Mayotte_gamb_2011


In [15]:
ag3.plot_heterozygosity(
    sample="AP0007-C", 
    region="3R", 
    site_mask="gamb_colu", 
    window_size=10_000,
);

Compute heterozygous genotypes:   0%|          | 0/350 [00:00<?, ?it/s]

We can see from this plot, that the demography of the Mayotte population is likely different from any we have looked at so far. Though the peak heterozygosity across the 3R contig is ~1.5%, there is much more spread, with most windows falling between 1.5% and 0.5% heterozygosity. Interpretations of this result could be that the island population was founded some time ago, and heterozygosity is returning to levels like we saw on the mainland _e.g._ Burkina Faso, and/or this is the stable level of heterozygosity for a small population size of _Anopheles_ mosquitoes.

**Exercise 1 (English)**

We have sampled another island population in Ag3.0, Bioko (part of Equatorial Guinea). Though, unlike Mayotte, Bioko is only ~30km away from mainland Africa. 

* Uncomment the code below and run a heterozygosity analysis for a sample from Bioko.
* How does this individual compare to the sample from Mayotte we looked at?
* Can you suggest why they might look different?

**Exercice 1 (Français)**

Nous avons des échantillons provenant d’une autre population insulaire dans Ag3.0: Bioko (qui fait partie de la Guinée-Équatoriale). Toutefois, contrairement à Mayotte, Bioko ne se trouve qu’à ~30 km du continent africain.

* Décommenter le code ci-dessous et effectuer une analyse de l’hétérozygosité pour un échantillon venant de Bioko.
* En quoi est-ce que cet individu est différent de celui que nous avons regardé pour Mayotte?
* Pouvez-vous proposer une explication pour ces différences?


In [16]:
# sample_df.query("location == 'Bioko'")

In [17]:
# ag3.plot_heterozygosity(
#     sample="???", 
#     region="3R", 
#     site_mask="gamb_colu", 
#     window_size=10_000,
# );

## Heterozygosity in an atypical wild population 

One of the reasons for running population genetic analyses like heterozygosity, is to detect populations that don't act as we expect them to, as this means we might be missing something about their demography which could be relevant to vector control. Samples of the putative cryptic species gcx3, collected in Kenya, have strange patterns of heterozygosity for wild samples, let's take a look and think about what the causes could be.

In [18]:
sample_df.query("taxon == 'gcx3' and country == 'Kenya'").head()

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,...,aim_species,country_iso,admin1_name,admin1_iso,admin2_name,taxon,cohort_admin1_year,cohort_admin1_month,cohort_admin2_year,cohort_admin2_month
1868,AK0049-C,KIL38,Janet Midega,Kenya,Kilifi,2012,-1,-3.511,39.909,F,...,intermediate_gambiae_coluzzii,KEN,Kilifi,KE-14,Kilifi,gcx3,KE-14_gcx3_2012,KE-14_gcx3_2012,KE-14_Kilifi_gcx3_2012,KE-14_Kilifi_gcx3_2012
1869,AK0050-C,KIL39,Janet Midega,Kenya,Kilifi,2012,-1,-3.511,39.909,F,...,intermediate_gambiae_coluzzii,KEN,Kilifi,KE-14,Kilifi,gcx3,KE-14_gcx3_2012,KE-14_gcx3_2012,KE-14_Kilifi_gcx3_2012,KE-14_Kilifi_gcx3_2012
1870,AK0051-C,KIL40,Janet Midega,Kenya,Kilifi,2012,-1,-3.511,39.909,F,...,intermediate_gambiae_coluzzii,KEN,Kilifi,KE-14,Kilifi,gcx3,KE-14_gcx3_2012,KE-14_gcx3_2012,KE-14_Kilifi_gcx3_2012,KE-14_Kilifi_gcx3_2012
1871,AK0052-C,KIL41,Janet Midega,Kenya,Kilifi,2012,-1,-3.511,39.909,F,...,intermediate_gambiae_coluzzii,KEN,Kilifi,KE-14,Kilifi,gcx3,KE-14_gcx3_2012,KE-14_gcx3_2012,KE-14_Kilifi_gcx3_2012,KE-14_Kilifi_gcx3_2012
1875,AK0062-C,KIL47,Janet Midega,Kenya,Kilifi,2012,-1,-3.511,39.909,F,...,gambiae,KEN,Kilifi,KE-14,Kilifi,gcx3,KE-14_gcx3_2012,KE-14_gcx3_2012,KE-14_Kilifi_gcx3_2012,KE-14_Kilifi_gcx3_2012


In [19]:
ag3.plot_heterozygosity(
    sample="AK0050-C", 
    region="3R", 
    site_mask="gamb_colu", 
    window_size=10_000,
);

Compute heterozygous genotypes:   0%|          | 0/350 [00:00<?, ?it/s]

So what is going on here? This wild caught sample has long __runs of homozygosity__ - regions where heterozygosity is close to zero - more so than even some of our lab colony samples. It looks like this individual has come from a wild population that has been through a severe bottleneck recently. But what might cause that in a natural population?

One possibility is that a vector control campaign that ran in Kenya prior to the date of collection was so successful that it rapidly and drastically reduced the population size of these mosquitoes. Unfortunately, this is just conjecture as we don't have longitudinal samples (before, during, after the control campaign) to test this hypothesis. 
  
Fortunately many of our sample sets going forward do contain longitudinally sampled mosquitoes, some in conjunction with vector control roll-out projects. Monitoring heterozygosity with these samples could potentially indicate if a control method _e.g._ next-generation bednets, affects mosquito __population sizes__ and thus provide an indication of its effectiveness.

**Exercise 2 (English)**

Another location where we have collected samples with atypical diversity statistics is Tanzania. Uncomment the code below and analyse the hetereozygosity of a gcx3 sample from Tanzania.

* Does your Tanzanian sample's heterozygosity differ from the sample from Kenya we looked at above? If so, how is it different?

**Exercice 2 (Français)**

Un autre lieu où nous avons des échantillons avec des statistiques de la diversité génétiques atypiques est la Tanzanie. Décommenter le code ci-dessous et analyser l’hétérozygotie d’un échantillon gcx3 tanzanien.

* Est-ce que l’hétérozygotie de votre échantillon tanzanien est différente de celle de l’échantillon kenyan que nous avons regardé en premier? Si oui, comment?

In [20]:
# sample_df.query("taxon == 'gcx3' and country == 'Tanzania'")

In [21]:
# ag3.plot_heterozygosity(
#     sample="???", 
#     region="3R", 
#     site_mask="gamb_colu", 
#     window_size=10_000,
# );

## Runs of homozygosity (ROH)

Plotting heterozygosity across individual genomes provides a great visual way to investigate population demography. For example, plotting heterozygosity across 3R suggests that the island population of Mayotte differs from mainland populations, however, it doesn't tell us exactly how it differs or by how much. 
  
One analysis that provides a way of quantifying heterozygosity, or rather the opposite side of the same coin - homozygosity, is to measure __runs of homozygosity__ or ROH. As we saw in our Kenyan heterozygosity plot above, runs of homozygosity are where we see multiple windows in row with near-zero heterozygosity (a.k.a. homozygosity).
  
To infer and measure these homozygous regions we can use a hidden markov model (HMM). HMMs were explained in [workshop 2, module 3](https://anopheles-genomic-surveillance.github.io/workshop-2/module-3-cnv-calling.html), where it was used to detect copy number variation from coverage data. But briefly, HMM is an algorithm that allows large, noisy datasets (such as heterozygosity), to be quickly scanned for patterns (such as runs of homozygosity).

For inferring ROH using an HMM, we need to install a package called `pomegranate`. If we are running this analysis on Google Colab, then we can use a quick install.

In [22]:
# quick install for colab
!pip install -q https://storage.googleapis.com/vo_agam_release/reference/training-images/workshop-5/pomegranate-0.14.8-cp37-cp37m-linux_x86_64.whl

However, if you are running this analysis somewhere other than colab, you'll need a full install of that package which will take a few minutes to build.

In [23]:
# Use ...
# !pip install pomegranate
# ... if running somewhere other than colab.

To infer and visualise ROH, we can use the function `plot_roh()`. Let's have a look at what parameters we need to specify for the `plot_roh()` function.

In [24]:
ag3.plot_roh?

Let's now plot ROH for the Kenyan sample we looked at earlier.

In [25]:
ag3.plot_roh(
    sample="AK0050-C", 
    region="3R", 
    site_mask="gamb_colu", 
    window_size=10_000,
);

Compute heterozygous genotypes:   0%|          | 0/350 [00:00<?, ?it/s]

The ROH analysis starts with the heterozygosity data that we plotted earlier and the top panel of the plot here is identical to the output of the `plot_heterozygosity()` function. The results from the HMM ROH analysis have been plotted below this, where we can see 23 blocks of ROH have been detected.

To access a Pandas DataFrame of the underlying ROH data, we can use another function, `roh_hmm`, with exactly the same parameters as before. The output DataFrame shows us the start, end and length of each ROH inferred by the HMM.

In [26]:
roh_df = ag3.roh_hmm(
    sample="AK0050-C", 
    region="3R", 
    site_mask="gamb_colu", 
    window_size=10_000,
)
roh_df

Compute heterozygous genotypes:   0%|          | 0/350 [00:00<?, ?it/s]

,sample_id,contig,roh_start,roh_stop,roh_length,roh_is_marginal
0,AK0050-C,3R,180,420337,420157,True
1,AK0050-C,3R,432202,1690670,1258468,False
2,AK0050-C,3R,1723873,2003686,279813,False
3,AK0050-C,3R,6108642,6123679,15037,False
4,AK0050-C,3R,11288732,12567018,1278286,False
5,AK0050-C,3R,12580510,14932888,2352378,False
6,AK0050-C,3R,17090690,17115221,24531,False
7,AK0050-C,3R,18307285,18605951,298666,False
8,AK0050-C,3R,19013506,19039488,25982,False
9,AK0050-C,3R,19451384,20434062,982678,False


In the [Ag1000G phase 1 paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6026373/) we computed ROH for all samples, allowing us to plot and compare them.
  
In the plot, each marker represents an individual mosquito. The Y-axis is the count of runs of homozygosity and the X-axis is the fraction of the genome composed of runs of homozygosity. Because these kind of data are inherently noisy, we filtered the ROH to only include those longer than 100,000 bp (100 kbp).

Lab colony parents are shown in black. We can see how most of them have lots of runs of homozygosity which make up much of their genomes, while most wild caught cohorts have much smaller fractions of their genomes made up of ROH.
  

<img src="https://storage.googleapis.com/vo_agam_release/reference/training-images/workshop-5/roh_scatter.svg" width="500 px"/>

**Exercise 3 (English)**

- What country might the grey samples come from (Hint: we've looked at strange patterns there today).
- Why might two of the lab colony parents (labelled "G") have both low ROH count and fraction ROH?

**Exercice 3 (Français)**

* De quel pays peuvent venir les échantillons gris du diagramme ci-dessus? (Indice: nous avons observé des motifs étranges dans ce pays aujourd’hui)
* Pourquoi est-ce que deux des parents venant de colonies (ceux étiquetés “G”) ont un compte des ROHs et une fraction de ROH si faibles?


Let's run our Kenyan sample through the same process we used to create the plot above.
 
First, filter the ROH DataFrame to just include runs of homozygosity longer than 100 kbp.

In [27]:
roh_df_100kb = roh_df.query("roh_length > 100_000")
roh_df_100kb

,sample_id,contig,roh_start,roh_stop,roh_length,roh_is_marginal
0,AK0050-C,3R,180,420337,420157,True
1,AK0050-C,3R,432202,1690670,1258468,False
2,AK0050-C,3R,1723873,2003686,279813,False
4,AK0050-C,3R,11288732,12567018,1278286,False
5,AK0050-C,3R,12580510,14932888,2352378,False
7,AK0050-C,3R,18307285,18605951,298666,False
9,AK0050-C,3R,19451384,20434062,982678,False
10,AK0050-C,3R,20449042,21095747,646705,False
11,AK0050-C,3R,25613447,28241877,2628430,False
12,AK0050-C,3R,28522124,29437036,914912,False


Now we can count the number of runs of homozygosity longer that 100 kbp...

In [28]:
roh_count = len(roh_df_100kb)
roh_count

18

...and, calculate the fraction ROH (fraction of the genome composed of ROH longer than 100_000 bp), by summing the "roh_length" column, then dividing this by the total length of the region.

In [29]:
roh_total_length = roh_df_100kb["roh_length"].sum()
roh_total_length

33142748

In [30]:
froh = roh_total_length / len(ag3.genome_sequence(region="3R"))
froh

0.6229759752712953

So, we can see that this individual is incredibly inbred for a wild caught mosquito, over 60% of the 3R chromosome arm is composed of long runs of homozygosity!

N.B., our ROH count will be different from the result shown for this individual in the Ag1000G phase 1 paper, because the paper combines ROH across multiple chromosomes, whereas here we have just calculated it across a single chromosome arm.

**Exercise 4 (English)**

Choose another sample from any mosquito population, and compute the number of ROH (`roh_count`) and fraction of genome in a ROH (`froh`) using only ROH larger than 100 kbp.

**Exercice 4 (Français)**

Choisissez un autre échantillon de n'importe quelle population de moustiques et calculez le nombre de ROH (`roh_count`) et la fraction de génome dans un ROH (`froh`) en utilisant uniquement des ROH supérieur à 100 kbp.

## Well done!

* In this module we have learnt about using heterozygosity and runs of homozygosity as tools for understanding mosquito population demography.
* Run, plotted and interpreted heterozygosity across regions of the genome.
* Run, plotted and interpreted ROH analysis.

## Exercises

Open this notebook in Google Colab, copy to your drive, clear outputs and run it for yourself from top to bottom. As you run through the notebook, cell by cell, think about what each cell is doing.

Have go at the practical exercises, but please don't worry if you don't have time to do them all during the practical session, and please ask the teaching assistants for help if you are stuck.

Hint: To open the notebook in Google Colab, click the rocket icon at the top of the page, then select “Colab” from the drop-down menu.